# INTRODUCTION

## Contexte du Projet

## Mise en place de PySpark

In [1]:
# Installation de pyspark
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTANT : PENSEZ A INSTALLER JAVA - winget install --exact --id EclipseAdoptium.Temurin.11.JDK

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test PySpark") \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

print(spark.version)

3.5.4


## Import et traitement des datasets

In [3]:
# Import des datasets ratings.csv & movies.csv
ratings = spark.read.csv("ml-32m/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("ml-32m/movies.csv", header=True, inferSchema=True)


In [4]:
# On Affiche les les 10 premières lignes des deux datasets
ratings.show(10)
movies.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     17|   4.0|944249077|
|     1|     25|   1.0|944250228|
|     1|     29|   2.0|943230976|
|     1|     30|   5.0|944249077|
|     1|     32|   5.0|943228858|
|     1|     34|   2.0|943228491|
|     1|     36|   1.0|944249008|
|     1|     80|   5.0|944248943|
|     1|    110|   3.0|943231119|
|     1|    111|   5.0|944249008|
+------+-------+------+---------+
only showing top 10 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|

In [5]:
# On affiche le schéma des deux datasets
ratings.printSchema()
movies.printSchema()



root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [6]:
# On regarde le nombre de lignes de chaque dataset
print("Nombre de lignes dans ratings : ", ratings.count())
print("Nombre de lignes dans movies : ", movies.count())

Nombre de lignes dans ratings :  32000204
Nombre de lignes dans movies :  87585


In [7]:
# On regarde le nombre de valeurs manquantes dans chaque dataset
from pyspark.sql.functions import isnan, when, count, col

ratings.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ratings.columns]).show()
movies.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in movies.columns]).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [8]:
# On regarde le nombre de valeurs distinctes dans chaque dataset
ratings.agg(*(count(col(c)).alias(c) for c in ratings.columns)).show()
movies.agg(*(count(col(c)).alias(c) for c in movies.columns)).show()

+--------+--------+--------+---------+
|  userId| movieId|  rating|timestamp|
+--------+--------+--------+---------+
|32000204|32000204|32000204| 32000204|
+--------+--------+--------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|  87585|87585| 87585|
+-------+-----+------+



In [12]:
# On regarde les statistiques descriptives des colonnes rating et timestamp du dataset ratings
ratings.describe(["rating", "timestamp"]).show()


+-------+------------------+--------------------+
|summary|            rating|           timestamp|
+-------+------------------+--------------------+
|  count|          32000204|            32000204|
|   mean|3.5403956487277393|1.2752411995732636E9|
| stddev|1.0589862139452866|2.5616297595059577E8|
|    min|               0.5|           789652004|
|    max|               5.0|          1697164147|
+-------+------------------+--------------------+



In [ ]:
# Quels sont les films les mieux notés en moyenne 
from pyspark.sql.functions import avg

ratings.groupBy("movieId").agg(avg("rating").alias("avg_rating")).orderBy("avg_rating", ascending=False).show()

+-------+----------+
|movieId|avg_rating|
+-------+----------+
| 238034|       5.0|
| 268024|       5.0|
| 284573|       5.0|
| 159904|       5.0|
| 266684|       5.0|
| 137048|       5.0|
| 289249|       5.0|
| 232745|       5.0|
| 206168|       5.0|
| 248742|       5.0|
| 136876|       5.0|
| 288987|       5.0|
| 133626|       5.0|
| 270306|       5.0|
| 213177|       5.0|
| 265044|       5.0|
| 225327|       5.0|
| 137052|       5.0|
| 221334|       5.0|
| 279542|       5.0|
+-------+----------+
only showing top 20 rows



In [ ]:
# Quels sont les films les plus populaires 
ratings.groupBy("movieId").count().orderBy("count", ascending=False).show()

Py4JJavaError: An error occurred while calling o242.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 172) (10.10.7.110 executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed

## Modelisation avec Spark MLlib : ALS

## Recommandation basée sur le contenu

## Recommandation basée sur les proximités utilisateurs (KNN)

## Evaluation des approches de recommandation